In [1]:
%%capture
%run validate_partner_manifest_dev.ipynb

In [2]:
def validate_bioscan(fn, template_fn='../data/BIOSCAN_Manifest_V2.0_20221017.xlsx', 
                     verbose=False, version='2.0', 
                     samples_sheet='TAB 2 Metadata Entry', contrib_sheet='TAB 1 Contributors'):
    '''
    BIOSCAN partner manifest validation
    Validation follows the order of columns order in data entry sheet
    '''

    setup_logging(verbose=verbose)

    logging.info(f'# started validate_bioscan_partner_manifest_v.{version}')
    logging.warning(f'# manifest {fn}')

    # read data
    df = get_data(fn, sheet=samples_sheet)
    df = remove_trailing_spaces(df)
    
    # read NCBI taxonomy
    ncbi = ete3.NCBITaxa()
    
    # prepare for validation
    template_df = get_data(template_fn, sheet='TAB 2 Metadata Entry')
    check_columns(df, template_df)
    valid_dict = get_valid_dict(template_fn, validation_sheet='TAB 4 DO NOT EDIT - Data Valida')

    # orange cols
    # exclude empty series
    df = validate_series(df)
    validate_freetext('CATCH_LOT', df, na_values=[])
    df = validate_plates_wells(df, 'RACK_OR_PLATE_ID', 'TUBE_OR_WELL_ID', bioscan=True)
    # check and exclude blanks
    is_blank = check_blanks(df)
    validate_values('ORGANISM_PART', df, valid_dict, sep='|')
    validate_values('PRESERVATIVE_SOLUTION', df, valid_dict)
    # columns below validated for non-blank samples only
    validate_values('BOTTLE_DIRECTION', df[~is_blank], valid_dict)
    date_coll = validate_date('DATE_OF_COLLECTION', df[~is_blank], na_values=['NOT_COLLECTED',''])
    # COUNTRY_OF_COLLECTION, DECIMAL_LATITUDE, DECIMAL_LONGITUDE
    validate_country_and_coordinates(df[~is_blank], fn, na_values=[''], bioscan=True)
    # COLLECTION_LOCATION not checked
    
    # purple cols - valiated for non-blank samples
    # WHAT_3_WORDS not checked
    validate_time('TIME_OF_COLLECTION', df[~is_blank])
    validate_time_period('DURATION_OF_COLLECTION', df[~is_blank], na_values=['NOT_COLLECTED'])
    validate_values('COLLECTION_METHOD', df[~is_blank], valid_dict)
    date_plat = validate_date('DATE_OF_PLATING', df[~is_blank], na_values=['NOT_COLLECTED',''])
    compare_dates(before=date_coll, after=date_plat)
    # taxonomy validation adds taxid columns to original dataframe
    df = validate_taxonomy(df, ncbi, anospp=False, na_values = [''])
    validate_values('SPECIMEN_IDENTITY_RISK', df[~is_blank], valid_dict, na_values=[''])
    validate_specimen_id_risk(df)
    validate_values('LIFESTAGE', df[~is_blank], valid_dict, na_values=[''])
    validate_values('SEX', df[~is_blank], valid_dict, na_values=[''])
    validate_values('SORTING_SOLUTION_USED', df[~is_blank], valid_dict)
    validate_values('CATCH_BOTTLE_TEMPERATURE_STORAGE', df[~is_blank], valid_dict)
    validate_values('PLATE_TEMPERATURE_STORAGE', df[~is_blank], valid_dict)
    
    # white cols - validated for all samples
    validate_freetext('MORPHOSPECIES_DESCRIPTION', df)
    validate_freetext('DESCRIPTION_OF_COLLECTION_METHOD', df)
    validate_freetext('HABITAT', df)
    validate_freetext('PRESERVATION_APPROACH', df)
    # TODO check if STS will need something here
    validate_freetext('COLLECTOR_SAMPLE_ID', df)
    validate_freetext('VOUCHER_ID', df)
    validate_float('ELEVATION', df, na_values=[''])
    validate_freetext('OTHER_INFORMATION', df)
    validate_freetext('MISC_METADATA', df)
    # IDENTIFIED_BY not checked
    # IDENTIFIER_AFFILIATION not checked
    
    validate_contributors(fn, contrib_sheet=contrib_sheet)
        
    logging.info('# ended validate_bioscan_partner_manifest_v.{}'.format(version))

    return df

# fn = '../../results/partner_manifests/IRD-Neandersquito_T222Amplicon_Manifest_V2.0.xlsx'
df = validate_bioscan(fn='../data/BIOSCAN_Manifest_V2.0_20221017.xlsx', 
                      template_fn='../data/BIOSCAN_Manifest_V2.0_20221017.xlsx', 
                      verbose=True,
                      samples_sheet='TAB 5 DO NOT EDIT - TEST Met')

[INFO] # started validate_bioscan_partner_manifest_v.2.0
[WARNING] # manifest ../data/BIOSCAN_Manifest_V2.0_20221017.xlsx
[INFO] reading data from '../data/BIOSCAN_Manifest_V2.0_20221017.xlsx'
[INFO] reading data from '../data/BIOSCAN_Manifest_V2.0_20221017.xlsx'
[INFO] checking manifest columns against template
[INFO] extracting value validation data from '../data/BIOSCAN_Manifest_V2.0_20221017.xlsx'
[INFO] validating SERIES
[INFO] validating freetext chars in CATCH_LOT
[INFO] validating RACK_OR_PLATE_ID and TUBE_OR_WELL_ID
[WARNING] found PLATE_ONLY plate SANG_001, expanding to well-level
[INFO] found 288 samples across 3 plates
[INFO] Checking and excluding blank samples
[WARNING] 14 blanks located across 3 plates, 274 samples of 288 left for downstream analysis
[INFO] validating values in column 'ORGANISM_PART'
[INFO] validating values in column 'PRESERVATIVE_SOLUTION'
[INFO] validating values in column 'BOTTLE_DIRECTION'
[INFO] validating date column 'DATE_OF_COLLECTION'
[INFO] ex

In [3]:
raise Exception('Setup complete')

Exception: Setup complete

In [ ]:
fn = '../results/20221018_biosc_test/NBGW-2206-Manifest-V2.0.xlsx'
df = validate_bioscan(fn, samples_sheet='Metadata Entry', contrib_sheet='TAB1 Contributors')